In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
data = pd.read_csv('../traces/fib30', header=None)
data.columns = ['branch_id', 'taken']
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4241389 entries, 0 to 4241388
Data columns (total 2 columns):
branch_id    int64
taken        int64
dtypes: int64(2)
memory usage: 64.7 MB


## 8192 Entry BHT (13-bit)

In [3]:
class BHT_core:
    def __init__(self, bits=13):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.bool)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = value
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]

In [4]:
class BHT_2_core:
    def __init__(self, bits=12):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros(2**self.bits, dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 3,
                       (2,0): 0, (2,1): 3,
                       (3,0): 2, (3,1): 3}
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id)] = self.states[self.table[self.key(branch_id)], value]
    
    def get(self, branch_id):
        return self.table[self.key(branch_id)]//2

In [5]:
class BHT_2_2_core:
    def __init__(self, bits=10):
        self.bits = bits
        self.mask = 2**self.bits - 1
        self.table = np.zeros([2**self.bits, 4], dtype=np.uint8)
        self.states = {(0,0): 0, (0,1): 1,
                       (1,0): 0, (1,1): 3,
                       (2,0): 0, (2,1): 3,
                       (3,0): 2, (3,1): 3}
        self.global_history = np.zeros(1, dtype=np.uint8)
    
    def key(self, branch_id):
        return branch_id & self.mask
        
    def update(self, branch_id, value):
        self.table[self.key(branch_id), self.global_history[0]] = self.states[self.table[self.key(branch_id), self.global_history[0]], value]
        self.global_history[0] = (self.global_history << 1 | value) % 4
    
    def get(self, branch_id):
        return self.table[self.key(branch_id), self.global_history[0]]//2

In [6]:
class BHT:
    def __init__(self, dataset, bht_type='BHT'):
        if bht_type.upper() == 'BHT':
            self.bht = BHT_core()
        elif bht_type.upper() == 'BHT_2':
            self.bht = BHT_2_core()
        elif bht_type.upper() == 'BHT_2_2':
            self.bht = BHT_2_2_core()
        self.codebase = dataset
    
    def run(self):
        bht = self.bht
        codebase = self.codebase
        total_predictions, mispredictions = (codebase.shape[0],0)
        start_time = time.clock()
        for i in range(codebase.shape[0]):
            branch_id = codebase.iloc[i].branch_id
            taken = codebase.iloc[i].taken
            if bht.get(branch_id) != taken:
                mispredictions = mispredictions + 1
            bht.update(branch_id,taken)
        end_time = time.clock()
        
        print('Total branches:        ', total_predictions)
        print('Unique branches:       ', codebase.branch_id.value_counts().shape[0])
        print('Correctly predicted:   ', total_predictions - mispredictions)
        print('Incorrectly predicted: ', mispredictions)
        print('Total time elaped:      %.2f seconds' % (end_time-start_time))

In [7]:
test = BHT(dataset=data, bht_type='bht')
test.run()

Total branches:         4241389
Unique branches:        491
Correctly predicted:    2480712
Incorrectly predicted:  1760677
Total time elaped:      1980.39 seconds


In [8]:
test = BHT(dataset=data, bht_type='bht_2')
test.run()

Total branches:         4241389
Unique branches:        491
Correctly predicted:    2649632
Incorrectly predicted:  1591757
Total time elaped:      1708.99 seconds


In [9]:
test = BHT(dataset=data, bht_type='bht_2_2')
test.run()

Total branches:         4241389
Unique branches:        491
Correctly predicted:    3227031
Incorrectly predicted:  1014358
Total time elaped:      1388.61 seconds
